In [1]:
import xarray
import pandas as pd 
import numpy as np

In [4]:
xa = xarray.open_dataset("tmin.nc", decode_times=False)
tmin = xa.to_dataframe().reset_index()
tmin.rename(columns={"X": "lon", "Y": "lat", "T": "months"}, inplace=True)
tmin.head()

,lon,months,lat,tmin
0,-92.525002,240.5,13.474999,NaN
1,-92.525002,240.5,13.525000,NaN
2,-92.525002,240.5,13.575000,NaN
3,-92.525002,240.5,13.624999,NaN
4,-92.525002,240.5,13.674999,NaN


In [5]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [6]:
def process_date(months):
    return dt.datetime(1960, 1, 1) + relativedelta(months=int(months))

tmin["date"] = tmin.months.map(process_date)

In [7]:
tmin["mes"] = tmin.date.map(lambda x: x.month)
tmin["anio"] = tmin.date.map(lambda x: x.year)
tmin = tmin[tmin.anio > 2001]
tmin.head()

,lon,months,lat,tmin,date,mes,anio
26928,-92.525002,504.5,13.474999,NaN,2002-01-01,1,2002
26929,-92.525002,504.5,13.525000,NaN,2002-01-01,1,2002
26930,-92.525002,504.5,13.575000,NaN,2002-01-01,1,2002
26931,-92.525002,504.5,13.624999,NaN,2002-01-01,1,2002
26932,-92.525002,504.5,13.674999,NaN,2002-01-01,1,2002


In [8]:
ndvi = pd.read_csv("ndvi-lluvia.csv")
ndvi.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat,lluvia_code,precipitacion
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211


In [9]:
adm2 = ndvi.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]
adm2.head()

/var/folders/sg/bv36hz0d5z36h52fqpd37kn80000gn/T/ipykernel_30263/2444717534.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  adm2 = ndvi.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]


,ADM2_PCODE,lat,lon
0,GT0100,14.435029,-90.548878
1,GT0101,14.635431,-90.479418
2,GT0102,14.564392,-90.465079
3,GT0103,14.554316,-90.346363
4,GT0104,14.793193,-90.372463


In [10]:
coord = tmin[["lon", "lat"]].drop_duplicates()
coord["tmin_code"] = range(len(coord))
coord.head()

,lon,lat,tmin_code
26928,-92.525002,13.474999,0
26929,-92.525002,13.525000,1
26930,-92.525002,13.575000,2
26931,-92.525002,13.624999,3
26932,-92.525002,13.674999,4


In [12]:
def lluvia_code(row):
    dist = np.sqrt((coord.lon - row.lon)**2 + (coord.lat - row.lat)**2)
    return int(coord.iloc[dist.argmin()]["tmin_code"])


adm2["tmin_code"] = adm2.apply(lluvia_code, axis=1)
adm2.head()

,ADM2_PCODE,lat,lon,tmin_code
0,GT0100,14.435029,-90.548878,4099
1,GT0101,14.635431,-90.479418,4205
2,GT0102,14.564392,-90.465079,4204
3,GT0103,14.554316,-90.346363,4510
4,GT0104,14.793193,-90.372463,4412


In [13]:
tmin = tmin.merge(coord, "left", on=["lat", "lon"])
ndvi = ndvi.merge(adm2[["ADM2_PCODE", "tmin_code"]], "left", on="ADM2_PCODE")
ndvi = ndvi.merge(tmin[["tmin_code", "mes", "anio", "tmin"]], "left", on=["tmin_code", "mes", "anio"])
ndvi.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat,lluvia_code,precipitacion,tmin_code,tmin
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487


In [15]:
ndvi = ndvi[ndvi.anio <= 2023]
ndvi.to_csv("ndvi-lluvia-tmin.csv", index=False)